In [3]:
!pip install opendatasets
!pip install tensorflow

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow_datasets as tfds
import opendatasets as od
import os

In [5]:
import tensorflow as tf
print("Cuda доступний:", tf.test.is_built_with_cuda())
print("GPU доступні:", tf.config.list_physical_devices('GPU'))

Cuda доступний: True
GPU доступні: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
od.download("https://www.kaggle.com/datasets/alessiocorrado99/animals10")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: illyawass
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10


In [8]:
dataset_path = 'animals10/raw-img'
classes = os.listdir(dataset_path)
print("Класи тварин:", classes)

Класи тварин: ['pecora', 'gallina', 'ragno', 'cavallo', 'cane', 'elefante', 'scoiattolo', 'farfalla', 'mucca', 'gatto']


In [9]:
NUM_EPOCHS = 25 #tride 10
BATCH_SIZE = 32 #tride: 32 (79%), 256(65%)
LEARNING_RATE = 0.001 #tride: 0.01, 0.1, 0.001
path_DS = 'animals10/raw-img'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [11]:
dataset = datasets.ImageFolder(root=path_DS, transform=transform)
num_classes = len(dataset.classes)
print(num_classes)

10


In [12]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [13]:
train_loader = DataLoader(train_dataset,pin_memory=True, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, pin_memory=True, batch_size=BATCH_SIZE, shuffle=False)

In [14]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # 32 →  64
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # 64 →  128
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # 128 → 256
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),  # 128 * 8 * 8 →  256 * 8 * 8  256 → 512
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)  # 256 → 512
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x


In [15]:
model = SimpleCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [16]:

def train_model():
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(NUM_EPOCHS):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f"Epoch {epoch+1}: Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        scheduler.step()

    return train_losses, train_accs, val_losses, val_accs

In [17]:
train_losses, train_accs, val_losses, val_accs = train_model()

Epoch 1/25: 100%|██████████| 655/655 [01:59<00:00,  5.50it/s]


Epoch 1: Train Loss: 2.0101, Train Acc: 0.2862, Val Loss: 1.7546, Val Acc: 0.3856


Epoch 2/25: 100%|██████████| 655/655 [01:56<00:00,  5.61it/s]


Epoch 2: Train Loss: 1.6826, Train Acc: 0.4169, Val Loss: 1.4853, Val Acc: 0.4849


Epoch 3/25: 100%|██████████| 655/655 [01:56<00:00,  5.61it/s]


...

Epoch 9: Train Loss: 1.1419, Train Acc: 0.6134, Val Loss: 1.1005, Val Acc: 0.6206


Epoch 10/25: 100%|██████████| 655/655 [01:59<00:00,  5.47it/s]


Epoch 10: Train Loss: 1.1211, Train Acc: 0.6200, Val Loss: 1.1127, Val Acc: 0.6102


Epoch 11/25:  71%|███████   | 466/655 [01:24<00:34,  5.49it/s]


KeyboardInterrupt: 

In [19]:
!pip3 install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 12.4 MB/s eta 0:00:00


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import optuna
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

def create_model(trial, num_classes):
    n_conv_layers = trial.suggest_int('n_conv_layers', 2, 4)

    initial_channels = trial.suggest_categorical('initial_channels', [32, 64, 128])

    conv_layers = []
    in_channels = 3
    out_channels = initial_channels

    for i in range(n_conv_layers):
        conv_layers.extend([
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
        in_channels = out_channels
        out_channels = trial.suggest_categorical(f'conv_{i}_out_channels', [64, 128, 256])

    def calculate_conv_output_size():
        x = torch.randn(1, 3, 128, 128)
        conv_seq = nn.Sequential(*conv_layers)
        return conv_seq(x).view(1, -1).size(1)

    fc_input_size = calculate_conv_output_size()

    n_fc_layers = trial.suggest_int('n_fc_layers', 1, 3)
    fc_layers = []

    prev_size = fc_input_size
    for i in range(n_fc_layers):
        fc_size = trial.suggest_categorical(f'fc_{i}_size', [128, 256, 512, 1024])
        fc_layers.extend([
            nn.Linear(prev_size, fc_size),
            nn.ReLU(),
            nn.Dropout(trial.suggest_float(f'dropout_{i}', 0.2, 0.5))
        ])
        prev_size = fc_size

    fc_layers.append(nn.Linear(prev_size, num_classes))

    class OptimizedCNN(nn.Module):
        def __init__(self):
            super(OptimizedCNN, self).__init__()
            self.conv_layers = nn.Sequential(*conv_layers)
            self.fc_layers = nn.Sequential(*fc_layers)

        def forward(self, x):
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)
            x = self.fc_layers(x)
            return x

    return OptimizedCNN()

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    path_DS = 'animals10/raw-img'
    dataset = datasets.ImageFolder(root=path_DS, transform=transform)
    num_classes = len(dataset.classes)

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_model(trial, num_classes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    best_val_accuracy = 0
    for epoch in range(10):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        best_val_accuracy = max(best_val_accuracy, val_accuracy)

        trial.report(val_accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_accuracy

def run_hyperparameter_optimization():
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3)
    )

    study.optimize(objective, n_trials=50)

    print("Найкращі гіперпараметри:", study.best_params)
    print("Найкраща точність:", study.best_value)

    return study.best_params, study.best_value


In [29]:

def objective(trial):
    print(f"\nПочаток тріалу {trial.number}")

    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    path_DS = 'animals10/raw-img'
    dataset = datasets.ImageFolder(root=path_DS, transform=transform)
    num_classes = len(dataset.classes)

    total_size = len(dataset)
    train_size = int(0.8 * total_size)
    val_size = total_size - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = create_model(trial, num_classes).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    epoch_accuracies = []
    for epoch in tqdm(range(10), desc=f"Навчання (LR: {lr}, Batch: {batch_size})"):
        # Навчання
        model.train()
        total_train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        epoch_accuracies.append(val_accuracy)

        trial.report(val_accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return max(epoch_accuracies)

def run_hyperparameter_optimization():
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=3),
        study_name='Animal Classification Hyperparameter Optimization'
    )

    optuna.logging.enable_default_handler()
    optuna.logging.enable_propagation()
    optuna.logging.set_verbosity(optuna.logging.INFO)

    study.optimize(objective, n_trials=10)

    try:
        plt.figure(figsize=(10, 6))
        plt.title('Еволюція найкращої точності')
        plt.plot(study.trials_dataframe()['value'])
        plt.xlabel('Номер тріалу')
        plt.ylabel('Точність')
        plt.tight_layout()
        plt.savefig('hyperparameter_optimization_history.png')
        plt.close()
    except Exception as e:
        print(f"Помилка при створенні графіку: {e}")

    print("\n--- Результати оптимізації ---")
    print("Найкращі гіперпараметри:", study.best_params)
    print("Найкраща точність:", study.best_value)

    try:
        with open('hyperparameter_optimization_report.txt', 'w') as f:
            f.write("Звіт оптимізації гіперпараметрів\n")
            f.write("=====================================\n")
            f.write(f"Найкращі гіперпараметри: {study.best_params}\n")
            f.write(f"Найкраща точність: {study.best_value}\n")
            f.write("\nДеталі всіх тріалів:\n")
            for trial in study.trials:
                f.write(f"Тріал {trial.number}: Параметри {trial.params}, Точність {trial.value}\n")
    except Exception as e:
        print(f"Помилка при створенні звіту: {e}")

    return study.best_params, study.best_value


Використання Optuna для пошуку гіперпараметрів

In [30]:
best_params, best_accuracy = run_hyperparameter_optimization()

[I 2025-03-26 16:28:11,543] A new study created in memory with name: Animal Classification Hyperparameter Optimization
INFO:optuna.storages._in_memory:A new study created in memory with name: Animal Classification Hyperparameter Optimization
<ipython-input-29-cfc7458a46bf>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)



Початок тріалу 0


Навчання (LR: 0.0007380687318695055, Batch: 64): 100%|██████████| 10/10 [15:01<00:00, 90.12s/it]
[I 2025-03-26 16:43:12,870] Trial 0 finished with value: 0.6617647058823529 and parameters: {'lr': 0.0007380687318695055, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 128, 'conv_0_out_channels': 64, 'conv_1_out_channels': 256, 'conv_2_out_channels': 256, 'conv_3_out_channels': 64, 'n_fc_layers': 1, 'fc_0_size': 256, 'dropout_0': 0.21577401776872118}. Best is trial 0 with value: 0.6617647058823529.
INFO:optuna.study.study:Trial 0 finished with value: 0.6617647058823529 and parameters: {'lr': 0.0007380687318695055, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 128, 'conv_0_out_channels': 64, 'conv_1_out_channels': 256, 'conv_2_out_channels': 256, 'conv_3_out_channels': 64, 'n_fc_layers': 1, 'fc_0_size': 256, 'dropout_0': 0.21577401776872118}. Best is trial 0 with value: 0.6617647058823529.



Початок тріалу 1


Навчання (LR: 0.00010370098997556227, Batch: 64): 100%|██████████| 10/10 [15:00<00:00, 90.03s/it]
[I 2025-03-26 16:58:13,254] Trial 1 finished with value: 0.6873567608861727 and parameters: {'lr': 0.00010370098997556227, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 256, 'conv_1_out_channels': 128, 'conv_2_out_channels': 64, 'conv_3_out_channels': 64, 'n_fc_layers': 2, 'fc_0_size': 512, 'dropout_0': 0.24734744921424534, 'fc_1_size': 1024, 'dropout_1': 0.24715336762524887}. Best is trial 1 with value: 0.6873567608861727.
INFO:optuna.study.study:Trial 1 finished with value: 0.6873567608861727 and parameters: {'lr': 0.00010370098997556227, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 256, 'conv_1_out_channels': 128, 'conv_2_out_channels': 64, 'conv_3_out_channels': 64, 'n_fc_layers': 2, 'fc_0_size': 512, 'dropout_0': 0.24734744921424534, 'fc_1_size': 1024, 'dropout_1': 0.24715336762524887}. Best is trial 1 with

...


Початок тріалу 8


Навчання (LR: 0.002162924982376065, Batch: 64):  30%|███       | 3/10 [06:05<14:12, 121.79s/it]
[I 2025-03-26 18:08:45,087] Trial 8 pruned. 
INFO:optuna.study._optimize:Trial 8 pruned. 



Початок тріалу 9


Навчання (LR: 0.0006177692135814439, Batch: 16): 100%|██████████| 10/10 [14:16<00:00, 85.60s/it]
[I 2025-03-26 18:23:01,301] Trial 9 finished with value: 0.7337662337662337 and parameters: {'lr': 0.0006177692135814439, 'batch_size': 16, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 128, 'conv_2_out_channels': 128, 'conv_3_out_channels': 256, 'n_fc_layers': 2, 'fc_0_size': 128, 'dropout_0': 0.21993023251827432, 'fc_1_size': 1024, 'dropout_1': 0.411053854481769}. Best is trial 2 with value: 0.7526737967914439.
INFO:optuna.study.study:Trial 9 finished with value: 0.7337662337662337 and parameters: {'lr': 0.0006177692135814439, 'batch_size': 16, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 128, 'conv_2_out_channels': 128, 'conv_3_out_channels': 256, 'n_fc_layers': 2, 'fc_0_size': 128, 'dropout_0': 0.21993023251827432, 'fc_1_size': 1024, 'dropout_1': 0.411053854481769}. Best is trial 2 with va


--- Результати оптимізації ---
Найкращі гіперпараметри: {'lr': 0.00032159231595668935, 'batch_size': 64, 'n_conv_layers': 4, 'initial_channels': 64, 'conv_0_out_channels': 128, 'conv_1_out_channels': 64, 'conv_2_out_channels': 256, 'conv_3_out_channels': 128, 'n_fc_layers': 1, 'fc_0_size': 1024, 'dropout_0': 0.37519808299731244}
Найкраща точність: 0.7526737967914439
